This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Reference frames

## Sample-detector geometry

The figure below shows the [sample reference frame](#detector-sample-geometry)
and the [detector reference frame](#detector-coordinates) used in kikuchipy, all
of which are right handed. In short, the sample reference frame ($X_s$, $Y_s$,
$Z_s$) is the one used by EDAX TSL, (RD, TD, ND), while the pattern center ($PC_x$,
$PC_y$, $PC_z$) is the one used by Bruker Nano, ($PC_x$, $PC_y$, $DD$).

<img id="detector-sample-geometry" src="../_static/image/reference_frames/sample_detector_geometry.png">

In **a**, the electron beam interacts with the sample in the source point, and
the shortest distance from this point to the detector is called the
projection/pattern center (PC). A part of the Kikuchi sphere resulting from the
beam-sample interaction is projected onto the flat detector in the gnomonic
projection, the EBSD pattern (EBSP). A gnomonic coordinate system $CS_g$, ($X_g$,
$Y_g$, $Z_g$) with (0, 0, 0) in the PC is defined for the detector plane. We
also define a detector coordinate system $CS_d$, ($X_d$, $Y_d$, $Z_d$), for the
detector plane, with (0, 0, 0) in the upper left corner. The projection center
coordinates ($PC_x$, $PC_y$, $PC_z$) are defined in this detector coordinate
system:
- $PC_x$ is measured from the left border of the detector in fractions of
  detector width.
- $PC_y$ is measured from the top border of the detector in fractions of
  detector height.
- $PC_z$ is the distance from the detector scintillator to the sample divided by
  pattern height.

Orientations are defined in the Bunge convention with respect to the sample
coordinate system $CS_s$, ($X_s$, $Y_s$, $Z_s$). The detector and sample viewed along
the microscope $X$ axis are shown in **b**, with the three coordinate systems
and the PC also shown. The scanned map is shown in **c**. Note the orientation
of $CS_s$ and the sample "Top": the map is scanned from the bottom of the sample
and upwards. Three tilt angles are defined: the sample tilt $\sigma$ shown in
**a** and **b**; the detector tilt $\theta$ shown in **b**; the azimuthal angle
$\omega$ which is defined as the sample tilt angle around the $X_s$ axis, shown
in a top view of the detector and sample along the microscope $Z$ axis in **d**.

<img id="detector-coordinates" src="../_static/image/reference_frames/gnomonic_detector_coordinates.png">

The above figure shows the EBSP in the
[sample reference frame figure](#sample-detector-geometry) **a** as viewed from
behind the screen towards the sample in (left) the gnomonic coordinate system
with its origin (0, 0) in the PC, and in (right) the detector coodinate system
with (0, 0) in the upper left pixel. The circles indicate the angular distance
from the PC in steps of $10^{\circ}$.

## The EBSD detector

All relevant parameters for the sample-detector geometry are stored in an
[kikuchipy.detectors.EBSDDetector](../reference/generated/kikuchipy.detectors.EBSDDetector.rst)
instance. Let's first import necessary libraries and a small Nickel EBSD test
data set

In [ ]:
# Exchange inline for notebook or qt5 (from pyqt) for interactive plotting
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp


s = kp.data.nickel_ebsd_small()  # Use kp.load("data.h5") to load your own data
s

Then we can define a detector with the same parameters as the one used to
acquire the small Nickel data set

In [ ]:
detector = kp.detectors.EBSDDetector(
    shape=s.axes_manager.signal_shape[::-1],
    pc=[0.421, 0.779, 0.505],
    convention="tsl",
    px_size=70,  # microns
    binning=8,
    tilt=0,
    sample_tilt=70,
)
detector

In [ ]:
detector.pc_tsl()

Above, the PC was passed in the EDAX TSL convention. Passing the PC in the
Bruker, Oxford, or EMsoft convention is also supported. The definitions of the
conventions are given in the
[EBSDDetector](../reference/generated/kikuchipy.detectors.EBSDDetector.rst) API
reference, together with the conversion from PC coordinates in the EDAX TSL,
Oxford, or EMsoft conventions to PC coordinates in the Bruker convention.

The PC coordinates in the EDAX TSL, Oxford, or EMsoft conventions can be
retreived via
[EBSDDetector.pc_tsl()](../reference/generated/kikuchipy.detectors.EBSDDetector.pc_tsl.rst),
[EBSDDetector.pc_oxford()](../reference/generated/kikuchipy.detectors.EBSDDetector.pc_oxford.rst),
and [EBSDDetector.pc_emsoft()](../reference/generated/kikuchipy.detectors.EBSDDetector.pc_emsoft.rst),
respectively. The latter requires the unbinned detector pixel size in microns
and the detector binning to be given upon initialization.

In [ ]:
detector.pc_emsoft()

The detector can be plotted to show whether the average PC is placed as
expected using
[EBSDDetector.plot()](../reference/generated/kikuchipy.detectors.EBSDDetector.plot.rst)
(see its docstring for a complete explanation of its parameters)

In [ ]:
detector.plot(pattern=s.inav[0, 0].data)

This will produce a figure similar to the right panel in the
[detector coordinates figure](#detector-coordinates) above, without the arrows
and colored labels.

Multiple PCs with a 1D or 2D navigation shape can be passed to the `pc`
parameter upon initialization, or can be set directly. This gives the detector
a navigation shape (not to be confused with the detector shape) and a navigation
dimension (maximum of two)

In [ ]:
detector.pc = np.ones([3, 4, 3]) * [0.421, 0.779, 0.505]
detector.navigation_shape

In [ ]:
detector.navigation_dimension

In [ ]:
detector.pc = detector.pc[0, 0]
detector.navigation_shape

<div class="alert alert-info">

Note

The offset and scale of HyperSpy’s `axes_manager` is fixed for a signal,
meaning that we cannot let the PC vary with scan position if we want to
calibrate the EBSD detector via the `axes_manager`. The need for a varying
PC was the main motivation behind the `EBSDDetector` class.
    
</div>

The left panel in the [detector coordinates figure](#detector-coordinates)
above shows the detector plotted in the gnomonic projection using
[EBSDDetector.plot()](../reference/generated/kikuchipy.detectors.EBSDDetector.plot.rst).
The 2D gnomonic coordinates ($x_g, y_g$) in $CS_g$ are defined in $CS_d$ are

$$
x_g = \frac{x_d}{z_d}, \qquad y_g = \frac{y_d}{z_d}.
$$

The detector bounds and pixel scale in this projection, per navigation point,
are stored with the detector

In [ ]:
detector.bounds

In [ ]:
detector.gnomonic_bounds

In [ ]:
detector.x_range

In [ ]:
detector.r_max  # Largest radial distance to PC

## Other software's reference frames

Other software use other reference frames. To aid in the conversion of a set of
observations, typically orientations, between softwares, the reference frames
used in other softwares are also shown here, represented to the best of the
contributors understanding.

<div class="alert alert-warning">

Warning

Reference frames of other software given here are based on instruction manuals
from the internet. Use with care, and double check whenever possible.
    
</div>

### EMsoft

<img id="sample-detector-geometry-emsoft" src="../_static/image/reference_frames/sample_detector_geometry_emsoft.png">

### Bruker

<img id="sample-detector-geometry-bruker" src="../_static/image/reference_frames/sample_detector_geometry_bruker.png">

### EDAX TSL

<img id="sample-detector-geometry-edax" src="../_static/image/reference_frames/sample_detector_geometry_edax.png">

### Oxford Instruments

<img id="sample-detector-geometry-oxford" src="../_static/image/reference_frames/sample_detector_geometry_oxford.png">